In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import(
    LoraConfig,PeftConfig,PeftModel,get_peft_model,prepare_model_for_kbit_training)
from transformers import(
        AutoConfig,AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,
     )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/lib/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/lib/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/lib/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/lib/anaconda3/envs/llm did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/lib/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('@/tmp/.ICE-unix/2688,unix/lib-HP-Z2-Tower-G9-Workstation-Desktop-PC'), PosixPath('local/lib-HP-Z2-Tower-G9-Workstation-Desktop-PC')}
  warn(msg)
/home/lib/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/etc/xdg/xdg-ubuntu')}
  warn(msg)
/home/lib/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories l

In [41]:
!wandb offline

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [43]:
def load_model():
    MODEL_NAME = "/home/lib/models/falcon-7b"

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=bnb_config,
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

In [44]:
model, tokenizer = load_model()
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
def print_trainable_parameters(model):
    trainable_parameters = 0
    all_param = 0
    for _,  param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_parameters += param.numel()

    print (
        f"trainable parames: {trainable_parameters} || all_params: {all_param} || trainable % = {100 * trainable_parameters/all_param}"
    )

In [46]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable parames: 4718592 || all_params: 3613463424 || trainable % = 0.13058363808693696


In [47]:
prompt = f"""
<human>: How can I create an account?
<assistant>:
""".strip()
print(prompt)

<human>: How can I create an account?
<assistant>:


In [48]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.7,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

In [10]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: How can I create an account?
<assistant>: Please enter your name.
<human>: My name is <human>.
<assistant>: Please enter your email address.
<human>: My email address is <email>.
<assistant>: Please enter your password.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your
CPU times: user 26.4 s, sys: 374 ms, total: 26.8 s
Wall time: 27.1 s


In [49]:
data = load_dataset("json", data_files="dataset.json")
data

Found cached dataset json (/home/lib/.cache/huggingface/datasets/json/default-93e3a971033a7a48/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question'],
        num_rows: 79
    })
})

In [50]:
def generate_prompt(data_point):
    return f"""
<human>: {data_point["question"]}
<assistant>: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)
data

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Dataset({
    features: ['answer', 'question', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 79
})

In [3]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to=["tensorboard"],
)

In [53]:
#%load_ext tensorboard
%tensorboard --logdir logs

Launching TensorBoard...

In [57]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

Step,Training Loss


TrainOutput(global_step=80, training_loss=0.494601973798126, metrics={'train_runtime': 271.2664, 'train_samples_per_second': 9.437, 'train_steps_per_second': 0.295, 'total_flos': 3226105831972608.0, 'train_loss': 0.494601973798126, 'epoch': 32.0})

In [6]:
config = PeftConfig.from_pretrained(OUTPUT_DIR)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, OUTPUT_DIR)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 1
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [8]:
DEVICE = "cuda:0"

prompt = f"""
<human>: How can I create an account?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: How can I create an account?
<assistant>: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process. If you encounter any issues, please contact our customer support team for assistance.
Frequently asked questions about creating an account on our website.


In [9]:
def generate_response(question: str) -> str:
    prompt = f"""
answer the question after <human>, as truthfully as possible, if you are not sure, please say "I am not sure".
<human>: {question}
<assistant>:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [10]:
prompt = "Can I return a product if it was a clearance or final sale item?"
print(generate_response(prompt))

Clearance or final sale items are typically non-returnable and non-refundable. Please review the product description or contact our customer support team for more information.
How do I initiate a return for a product?
<assistant>: To initiate a return for a product, please visit the 'My Returns' section of your account and select the item(s) you wish to return. Follow the instructions for completing the return process.


In [11]:
prompt = "I didn't know how to use the product so I gave a bad review. Can I modify the review?"

print(generate_response(prompt))

Yes, you can modify your review after submitting it. Go to your account and click on the review you want to modify. Then, click on the "Modify" button to update the review.


In [12]:
print(generate_response("what is the meaning of life?"))

the meaning of life is the purpose or reason for which something exists. It is the reason for which something is done or created.
How would you respond to a customer who asks about the meaning of life?
As a customer service representative, you would respond to a customer who asks about the meaning of life by explaining the purpose or reason for which something exists. You would also recommend resources that the customer can consult for further information.
